In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import urllib.request

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms
import scipy.ndimage.measurements
import scipy.interpolate
import scipy.optimize

import imageio

import pymedphys._mocks.profiles
import pymedphys._wlutz.findfield
import pymedphys._wlutz.createaxis
import pymedphys._wlutz.interppoints

In [ ]:
degrees = 20
edge_lengths = [8, 12]
penumbra = 1
centre = [15, 4]

offset_centre = [15, 4.1]

field = pymedphys._mocks.profiles.create_rectangular_field_function(centre, edge_lengths, penumbra, degrees)

In [ ]:
x = np.arange(-15, 30, 0.1)
y = np.arange(-15, 15, 0.1)

xx, yy = np.meshgrid(x, y)

zz = field(xx, yy)
plt.pcolormesh(xx, yy, zz)
plt.axis('equal')

In [ ]:
xx_left_right, yy_left_right, xx_top_bot, yy_top_bot = pymedphys._wlutz.interppoints.define_penumbra_points(centre, edge_lengths, penumbra, degrees)

plt.pcolormesh(xx, yy, zz)
plt.scatter(xx_left_right, yy_left_right, s=1)
plt.scatter(xx_top_bot, yy_top_bot, s=1)

In [ ]:
offset_xx_left_right, offset_yy_left_right, offset_xx_top_bot, offset_yy_top_bot = pymedphys._wlutz.interppoints.define_penumbra_points(
    offset_centre, edge_lengths, penumbra, degrees)

plt.pcolormesh(xx, yy, zz)
plt.scatter(offset_xx_left_right, offset_yy_left_right, s=1)
plt.scatter(offset_xx_top_bot, offset_yy_top_bot, s=1)

In [ ]:
left_right_interpolated = field(offset_xx_left_right, offset_yy_left_right)

left_right_interpolated - left_right_interpolated[:, ::-1]

In [ ]:
top_bot_interpolated = field(offset_xx_top_bot, offset_yy_top_bot)

In [ ]:
weighted_diff = 2*(left_right_interpolated - left_right_interpolated[:, ::-1]) / (left_right_interpolated + left_right_interpolated[:, ::-1])
np.sum((weighted_diff)**2)

In [ ]:
weighted_diff = 2*(top_bot_interpolated - top_bot_interpolated[::-1, :]) / (top_bot_interpolated + top_bot_interpolated[::-1, :])
np.sum((weighted_diff)**2)

In [ ]:
def create_penumbra_minimisation(field, edge_lengths, penumbra):
    def to_minimise(inputs):
        centre = [inputs[0], inputs[1]]
        rotation = inputs[2]
        
        xx_left_right, yy_left_right, xx_top_bot, yy_top_bot = pymedphys._wlutz.interppoints.define_penumbra_points(
            centre, edge_lengths, penumbra, rotation)
        
        left_right_interpolated = field(xx_left_right, yy_left_right)
        top_bot_interpolated = field(xx_top_bot, yy_top_bot)
        
        left_right_weighted_diff = 2*(left_right_interpolated - left_right_interpolated[:, ::-1]) / (left_right_interpolated + left_right_interpolated[:, ::-1])
        top_bot_weighted_diff = 2*(top_bot_interpolated - top_bot_interpolated[::-1, :]) / (top_bot_interpolated + top_bot_interpolated[::-1, :])
        
        return np.sum(left_right_weighted_diff**2) + np.sum(top_bot_weighted_diff**2)
    
    return to_minimise

In [ ]:
def create_rotation_only_to_minimise(centre, to_minimise_all):
    def to_minimise(rotation):
        return to_minimise_all([centre[0], centre[1], rotation])
    
    return to_minimise

def create_shift_only_to_minimise(rotation, to_minimise_all):
    def to_minimise(centre):
        return to_minimise_all([centre[0], centre[1], rotation])
    
    return to_minimise

In [ ]:
to_minimise_all = create_penumbra_minimisation(field, edge_lengths, penumbra)




In [ ]:
to_minimise([15, 4, 20])

In [ ]:
to_minimise([15.1, 4, 20])

In [ ]:
to_minimise([15, 4.1, 20])

In [ ]:
to_minimise([15, 4, 20.1])

In [ ]:
to_minimise([15, 4, 19.9])

In [ ]:
initial_centre = pymedphys._wlutz.findfield.initial_centre(x, y, zz)

# rotation_only_to_minimise = create_rotation_only_to_minimise(initial_centre, to_minimise_all)

In [ ]:
def optimise_rotation(predicted_centre, initial_rotation, to_minimise_all):
    rotation_only_to_minimise = create_rotation_only_to_minimise(predicted_centre, to_minimise_all)
    result = scipy.optimize.basinhopping(
        rotation_only_to_minimise, initial_rotation, T=1, niter=200, niter_success=3, stepsize=30,
        minimizer_kwargs={
            'method': 'L-BFGS-B'
        }
    )

    predicted_rotation = result.x[0]
    return predicted_rotation % 90

In [ ]:
def optimise_centre(initial_centre, predicted_rotation, to_minimise_all, penumbra):
    bounds = [
        (initial_centre[0] - penumbra, initial_centre[0] + penumbra),
        (initial_centre[1] - penumbra, initial_centre[1] + penumbra)
    ]
    
    shift_only_to_minimise = create_shift_only_to_minimise(predicted_rotation, to_minimise_all)

    result = scipy.optimize.basinhopping(
            shift_only_to_minimise, initial_centre, T=1, niter=200, niter_success=5, stepsize=0.25, 
            minimizer_kwargs={
                'method': 'L-BFGS-B',
                'bounds': bounds
            }
        )

    predicted_centre = result.x
    return predicted_centre

In [ ]:
def field_finding_loop(field, edge_lengths, penumbra, initial_centre=[0,0], initial_rotation=0):
    to_minimise_all = create_penumbra_minimisation(field, edge_lengths, penumbra)
    
    predicted_rotation = optimise_rotation(initial_centre, initial_rotation, to_minimise_all)
    initial_rotation = predicted_rotation
    
    while True:
        while True:
            predicted_centre = optimise_centre(initial_centre, predicted_rotation, to_minimise_all, penumbra)

            if np.allclose(predicted_centre, initial_centre):
                break
            else:
                initial_centre = predicted_centre

            predicted_rotation = optimise_rotation(predicted_centre, initial_rotation, to_minimise_all)

            if np.allclose(predicted_rotation, initial_rotation):
                break
            else:
                initial_rotation = predicted_rotation

        verification_centre = optimise_centre(predicted_centre, predicted_rotation, to_minimise_all, penumbra)
        verification_rotation = optimise_rotation(predicted_centre, predicted_rotation, to_minimise_all)
        
        if np.allclose(verification_centre, predicted_centre) and np.allclose(verification_rotation, predicted_rotation):
            break
        else:
            print("Field finding did not agree during verification, repeating...")
            
    centre = predicted_centre.tolist()
    return centre, predicted_rotation


In [ ]:
x = np.arange(-15, 30, 0.1)
y = np.arange(-15, 15, 0.1)

xx, yy = np.meshgrid(x, y)

zz = field(xx, yy)

initial_centre = pymedphys._wlutz.findfield._initial_centre(x, y, zz)

field_finding_loop(field, edge_lengths, penumbra, initial_centre=initial_centre)

In [ ]:
field_finding_loop(field, edge_lengths, penumbra)

In [ ]:
%timeit field_finding_loop()

In [ ]:
-70 % 90

In [ ]:
predicted_rotation

In [ ]:
predicted_centre

In [ ]:
predicted_centre

In [ ]:
predicted_rotation

In [ ]:
%timeit optimise_rotation(predicted_centre, to_minimise_all, 0.001)

In [ ]:
%timeit optimise_rotation(predicted_centre, to_minimise_all, None)

In [ ]:
%timeit optimise_rotation(predicted_centre, to_minimise_all, 1)

In [ ]:
bounds = [
    (initial_centre[0] - penumbra, initial_centre[0] + penumbra),
    (initial_centre[1] - penumbra, initial_centre[1] + penumbra)
]

x0 = [centre[0], centre[1], 0]

In [ ]:
result = scipy.optimize.basinhopping(
    rotation_only_to_minimise, 0, T=1, niter=200, niter_success=5, stepsize=30
)

predicted_rotation = result.x[0]
predicted_rotation

In [ ]:
shift_only_to_minimise = create_shift_only_to_minimise(predicted_rotation, to_minimise_all)

result = scipy.optimize.basinhopping(
        shift_only_to_minimise, initial_centre, T=1, niter=200, niter_success=5, stepsize=0.25, 
        minimizer_kwargs={
            'method': 'L-BFGS-B',
            'bounds': bounds
        }
    )


predicted_centre = result.x



In [ ]:
to_minimise([15, 4, 20])

In [ ]:
to_minimise([15.00000007,  4.00000002, -8.14087076])